In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import fastf1
from src.plotset import setup_plot
from fastf1 import plotting

setup_plot()

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
events_df = fastf1.get_event_schedule(2025,include_testing=False)

In [ ]:
pia, nor, ver = [], [], []

for i in range(1,13):
   
    race = fastf1.get_session(2025,i,'R')
    race.load(laps=False,telemetry=False,weather=False,messages=False)

    if i in [2,6]:
        sprint = fastf1.get_session(2025,i,'S')
        sprint.load(laps=False,telemetry=False,weather=False,messages=False)
    else:
        sprint = None
    
    pia.append(race.get_driver('PIA').Points + (sprint.get_driver('PIA').Points if sprint is not None else 0))
    nor.append(race.get_driver('NOR').Points + (sprint.get_driver('NOR').Points if sprint is not None else 0))
    ver.append(race.get_driver('VER').Points + (sprint.get_driver('VER').Points if sprint is not None else 0))

In [ ]:
standings_df = pd.DataFrame({'Round':range(13),
                             'PIA':[0] + pia,
                             'NOR':[0] + nor,
                             'VER':[0] + ver})

In [ ]:
standings_df[['PIA','NOR','VER']] = standings_df[['PIA','NOR','VER']].cumsum()

In [ ]:
standings_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_standings_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=standings_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=standings_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=standings_df.VER.values),2)})

smooth_standings_df

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

# Create empty line objects
drivers = ['PIA', 'NOR', 'VER']
lines = {}
marker_lines = {}

for drv in drivers:
    style = plotting.get_driver_style(identifier=drv, session=race, style=['color', 'linestyle'])
    
    # Line placement
    if drv=='PIA':
        style['linestyle'] = 'solid'
    elif drv=='NOR':
        style['color'] = "#CCFF00"
    line, = ax.plot([], [], label=drv, lw=3, **style)
    lines[drv] = line

    # Marker placement
    marker_line, = ax.plot([], [], lw=0, marker='o', color=style['color'], markersize=8)
    marker_lines[drv] = marker_line

# Axis labels and legend
ax.set_xticks(range(1, 13))
ax.set_xticklabels(ax.get_xticklabels())
#ax.set_xlabel('.',color="#00000000")
#ax.set_xlim([0,13])

ax.set_yticks(range(0,325,25))
ax.set_yticklabels(ax.get_yticklabels())
#ax.set_ylabel('Championship Points')
#ax.set_ylim([0,300])

ax.set_title('Driver Championship Points')

h, l = ax.get_legend_handles_labels()
ax.legend(handles=h,labels=['Oscar Piastri','Lando Norris','Max Verstappen'],fontsize=16,loc='upper left')

# Update function
def update(frame):
    ymin, ymax = 0, 25
    xmin, xmax = 0, 2

    for drv in drivers:
        line = lines[drv]
        xdata = smooth_standings_df['Round'][:frame]
        ydata = smooth_standings_df[drv][:frame]
        line.set_data(xdata, ydata)

        marker_line = marker_lines[drv]
        x_markers = xdata[xdata.isin(range(1, 13))]
        y_markers = ydata[xdata.isin(range(1, 13))]
        marker_line.set_data(x_markers, y_markers)

        # Update ymin and ymax based on current data
        if len(ydata) > 0:
            ymin = min(ymin, ydata.min())
            ymax = max(ymax, ydata.max())
        
        if len(xdata) > 0:
            xmin = min(xmin, xdata.min())
            xmax = max(xmax, xdata.max())

    # Add some padding so lines don’t touch axis edges
    padding = (ymax - ymin) * 0.25 if ymax > ymin else 10
    ax.set_ylim(ymin, ymax + padding)
    ax.set_xlim(xmin, xmax + 1)

    return list(lines.values())

# Create animation
ani = FuncAnimation(
    fig,
    update,
    frames=len(smooth_standings_df)+1,
    interval=30,
    blit=True,
    repeat=False
)

HTML(ani.to_jshtml())

# # Save the animation as MP4 video
# ani.save('./media/driver_standings_Long.mp4', writer='ffmpeg', fps=30, dpi=300, bitrate=8000)

In [ ]:
# Qualifying Results

pia, nor, ver = [], [], []

for i in range(1,13):
   
    quali = fastf1.get_session(2025,i,'Q')
    quali.load(laps=False,telemetry=False,weather=False,messages=False)
    
    pia.append(quali.get_driver('PIA').Position)
    nor.append(quali.get_driver('NOR').Position)
    ver.append(quali.get_driver('VER').Position)

quali_df = pd.DataFrame({'Round':range(13),
                         'PIA':[0] + pia,
                         'NOR':[0] + nor,
                         'VER':[0] + ver})

In [ ]:
quali_df

In [ ]:
poles_df = quali_df.copy()
poles_df[['PIA','NOR','VER']] = quali_df.drop('Round',axis=1)[quali_df == 1].fillna(0).cumsum()
poles_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_poles_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=poles_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=poles_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=poles_df.VER.values),2)})

smooth_poles_df

In [ ]:
avgQ_df = quali_df.copy()
avgQ_df.iloc[1:,1:] = avgQ_df[['PIA','NOR','VER']].drop(0).cumsum()
avgQ_df.iloc[1:,1:] = np.round(avgQ_df.iloc[1:,1:].div(avgQ_df.Round.iloc[1:],axis=0),2)
avgQ_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_avgQ_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgQ_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgQ_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgQ_df.VER.values),2)})

smooth_avgQ_df

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

bars = ax.bar(x=['PIA','NOR','VER'],height=smooth_avgQ_df.drop('Round',axis=1).iloc[0],color=['#FF8000','#CCFF00','#0600EF'])
ax.set_xticks(['PIA','NOR','VER'])
ax.set_xticklabels(['','',''])
ax.set_title('Average Qualifying Position',fontsize=16)

# Update bar heights
def update(frame):
    levels = smooth_avgQ_df.drop('Round',axis=1).iloc[frame]
    for bar, level in zip(bars,levels):
        bar.set_height(level)

    ymin, ymax = 0, 1
    if min(levels) != max(levels):
        ymin = 0
        ymax = max(levels) * 1.2
    ax.set_ylim(ymin,ymax)

    return bars

# Animate the chart
ani = FuncAnimation(
    fig, update, frames=len(smooth_avgQ_df),blit=True,interval=20,repeat=False
)

HTML(ani.to_jshtml())

# Save the animation as MP4 video
ani.save('./media/avgQPos.mp4', writer='ffmpeg', fps=30, dpi=300, bitrate=8000)

In [ ]:
# Race Results

pia, nor, ver = [], [], []

for i in range(1,13):
   
    race = fastf1.get_session(2025,i,'R')
    race.load(laps=False,telemetry=False,weather=False,messages=False)
    
    pia.append(race.get_driver('PIA').Position)
    nor.append(race.get_driver('NOR').Position)
    ver.append(race.get_driver('VER').Position)

race_df = pd.DataFrame({'Round':range(13),
                         'PIA':[0] + pia,
                         'NOR':[0] + nor,
                         'VER':[0] + ver})

In [ ]:
race_df

In [ ]:
wins_df = race_df.copy()
wins_df[['PIA','NOR','VER']] = race_df.drop('Round',axis=1)[race_df == 1].fillna(0).cumsum()
wins_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_wins_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=wins_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=wins_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=wins_df.VER.values),2)})

smooth_wins_df

In [ ]:
avgR_df = race_df.copy()
avgR_df.iloc[1:,1:] = avgR_df[['PIA','NOR','VER']].drop(0).cumsum()
avgR_df.iloc[1:,1:] = np.round(avgR_df.iloc[1:,1:].div(avgR_df.Round.iloc[1:],axis=0),2)
avgR_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_avgR_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgR_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgR_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgR_df.VER.values),2)})

smooth_avgR_df

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

bars = ax.bar(x=['PIA','NOR','VER'],height=smooth_avgR_df.drop('Round',axis=1).iloc[0],color=['#FF8000','#CCFF00','#0600EF'])
ax.set_xticks(['PIA','NOR','VER'])
ax.set_xticklabels(['','',''])
ax.set_title('Average Race Finish Position',fontsize=16)

# Update bar heights
def update(frame):
    levels = smooth_avgR_df.drop('Round',axis=1).iloc[frame]
    for bar, level in zip(bars,levels):
        bar.set_height(level)

    ymin, ymax = 0, 1
    if min(levels) != max(levels):
        ymin = 0
        ymax = max(levels) * 1.2
    ax.set_ylim(ymin,ymax)

    return bars

# Animate the chart
ani = FuncAnimation(
    fig, update, frames=len(smooth_avgR_df),blit=True,interval=20,repeat=False
)

HTML(ani.to_jshtml())

# Save the animation as MP4 video
ani.save('./media/avgRPos.mp4', writer='ffmpeg', fps=30, dpi=300, bitrate=8000)

In [ ]:
# Q-->R Results

pia, nor, ver = [], [], []

for i in range(1,13):
   
    race = fastf1.get_session(2025,i,'R')
    race.load(laps=False,telemetry=False,weather=False,messages=False)
    
    pia.append(- race.get_driver('PIA').Position + race.get_driver('PIA').GridPosition)
    nor.append(- race.get_driver('NOR').Position + race.get_driver('NOR').GridPosition)
    ver.append(- race.get_driver('VER').Position + race.get_driver('VER').GridPosition)

gains_df = pd.DataFrame({'Round':range(13),
                         'PIA':[0] + pia,
                         'NOR':[0] + nor,
                         'VER':[0] + ver})

In [ ]:
gains_df

In [ ]:
avgG_df = gains_df.copy()
avgG_df.iloc[1:,1:] = avgG_df[['PIA','NOR','VER']].drop(0).cumsum()
avgG_df.iloc[1:,1:] = np.round(avgG_df.iloc[1:,1:].div(avgG_df.Round.iloc[1:],axis=0),2)
avgG_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_avgG_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgG_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgG_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=avgG_df.VER.values),2)})

smooth_avgG_df

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

bars = ax.bar(x=['PIA','NOR','VER'],height=smooth_avgG_df.drop('Round',axis=1).iloc[0],color=['#FF8000','#CCFF00','#0600EF'])
ax.set_xticks(['PIA','NOR','VER'])
ax.set_xticklabels(['','',''])
ax.set_title('Average Positions Gained',fontsize=16)

# Update bar heights
def update(frame):
    levels = smooth_avgG_df.drop('Round',axis=1).iloc[frame]
    for bar, level in zip(bars,levels):
        bar.set_height(level)

    ymin, ymax = -0.04, 0.04
    if min(levels) != max(levels):
        swing = max(abs(min(levels)),abs(max(levels)))
        ymin, ymax = -swing*1.2, swing*1.2

    ax.set_ylim(ymin,ymax)

    return bars

# Animate the chart
ani = FuncAnimation(
    fig, update, frames=len(smooth_avgG_df),blit=True,interval=20,repeat=False
)

HTML(ani.to_jshtml())

# Save the animation as MP4 video
ani.save('../media/avgPosG.mp4', writer='ffmpeg', fps=30, dpi=300, bitrate=8000)

In [ ]:
# %Laps Led Results

pia, nor, ver, laps = [], [], [], []

for i in range(1,13):
   
    race = fastf1.get_session(2025,i,'R')
    race.load(laps=True,telemetry=False,weather=False,messages=False)
    
    pia.append(race.laps.pick_drivers('PIA').Position.value_counts().get(1,0))
    nor.append(race.laps.pick_drivers('NOR').Position.value_counts().get(1,0))
    ver.append(race.laps.pick_drivers('VER').Position.value_counts().get(1,0))
    laps.append(race.results.iloc[0].Laps)

laps_df = pd.DataFrame({'Round':range(13),
                         'PIA':[0] + pia,
                         'NOR':[0] + nor,
                         'VER':[0] + ver,
                         'laps': [0] + laps})

In [ ]:
laps_df.PIA = laps_df.PIA.astype(float)
laps_df.NOR = laps_df.NOR.astype(float)
laps_df.VER = laps_df.VER.astype(float)

In [ ]:
laps_df

In [ ]:
laps_percent_df = laps_df.copy()
laps_percent_df.iloc[1:,1:] = laps_percent_df[['PIA','NOR','VER','laps']].drop(0).cumsum()
laps_percent_df.iloc[1:,1:-1] = np.round(laps_percent_df.iloc[1:,1:-1].div(laps_percent_df.laps.iloc[1:],axis=0),2)*100
laps_percent_df

In [ ]:
rounds_data = np.linspace(0,12,601)
smooth_laps_percent_df = pd.DataFrame({'Round': np.round(rounds_data,2),
                                 'PIA': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=laps_percent_df.PIA.values),2),
                                 'NOR': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=laps_percent_df.NOR.values),2),
                                 'VER': np.round(np.interp(x=rounds_data,xp=range(0,13),fp=laps_percent_df.VER.values),2)})

smooth_laps_percent_df

In [ ]:
fig, ax = plt.subplots(figsize=(6,4))

bars = ax.bar(x=['PIA','NOR','VER'],height=smooth_laps_percent_df.drop('Round',axis=1).iloc[0],color=['#FF8000','#CCFF00','#0600EF'])
ax.set_xticks(['PIA','NOR','VER'])
ax.set_xticklabels(['','',''])
ax.set_title('% of Laps Led',fontsize=16)
ax.set_ylim(0,100)

# Update bar heights
def update(frame):
    levels = smooth_laps_percent_df.drop('Round',axis=1).iloc[frame]
    for bar, level in zip(bars,levels):
        bar.set_height(level)

    return bars

# Animate the chart
ani = FuncAnimation(
    fig, update, frames=len(smooth_laps_percent_df),blit=True,interval=20,repeat=False
)

HTML(ani.to_jshtml())

# Save the animation as MP4 video
ani.save('../media/avgLapLed.mp4', writer='ffmpeg', fps=30, dpi=300, bitrate=8000)